<a href="https://colab.research.google.com/github/joeynmt/joeynmt/blob/main/joey_v1_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Joey NMT v1 Demo

In this notebook, we'll train a Transformer model for translating between simple sentences in Esperanto (*epo*) and English (*eng*). You'll have the option to choose your own languages as well.

> **Note:** This notebook was initially created for Joey NMT v1.x.  

**Important:** Before you start, set runtime type to GPU.

Author: Julia Kreutzer

## Installation

Install JoeyNMT v1.5 via [pip](https://pypi.org/project/joeynmt/).

It works on PyTorch version 1.10.1. If that's not automatically installed, run the following command to install it.



```
!pip install torch==1.10.1+cu102 torchtext==0.11.1 -f https://download.pytorch.org/whl/torch_stable.html
```



In [2]:
!pip install -q git+https://github.com/joeynmt/joeynmt.git@1.5

     |████████████████████████████████| 116 kB 30.2 MB/s 
     |████████████████████████████████| 505 kB 54.9 MB/s 
     |████████████████████████████████| 103 kB 49.9 MB/s 
     |████████████████████████████████| 262 kB 72.9 MB/s 
     |████████████████████████████████| 843 kB 61.5 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 


# Data Preparation

We'll use English - Esperanto translations from the Tatoeba challenge. 

If you want to use a different language pair, you'll need to replace all instances of `eng` and `epo` language identifiers to your languages of choice. You can find all available languages [here](https://github.com/Helsinki-NLP/Tatoeba-Challenge/blob/master/Data.md). Note that if the training data size is larger, data preparation and training might take much longer than the example with only 400k sentence pairs.

## Download

In [3]:
!wget https://object.pouta.csc.fi/Tatoeba-Challenge/eng-epo.tar

--2022-09-04 07:56:20--  https://object.pouta.csc.fi/Tatoeba-Challenge/eng-epo.tar
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35747840 (34M) [application/x-tar]
Saving to: ‘eng-epo.tar’

eng-epo.tar         100%[===================>]  34.09M  8.18MB/s    in 4.2s    

2022-09-04 07:56:27 (8.18 MB/s) - ‘eng-epo.tar’ saved [35747840/35747840]



In [4]:
!tar -xvf  'eng-epo.tar'

data/eng-epo/
data/eng-epo/train.src.gz
data/eng-epo/dev.trg
data/eng-epo/train.id.gz
data/eng-epo/test.trg
data/eng-epo/test.id
data/eng-epo/dev.src
data/eng-epo/dev.id
data/eng-epo/test.src
data/eng-epo/train.trg.gz


In [5]:
!gunzip 'data/eng-epo/train.src.gz'
!gunzip 'data/eng-epo/train.trg.gz'

We'll only use a subset of dev and test data.

In [6]:
!mv 'data/eng-epo/train.src' 'data/eng-epo/train.eng'
!head -n 1000 'data/eng-epo/dev.src' > 'data/eng-epo/dev.eng'
!head -n 1000 'data/eng-epo/test.src' > 'data/eng-epo/test.eng'
!mv 'data/eng-epo/train.trg' 'data/eng-epo/train.epo'
!head -n 1000 'data/eng-epo/dev.trg' > 'data/eng-epo/dev.epo'
!head -n 1000 'data/eng-epo/test.trg' > 'data/eng-epo/test.epo'

The data is sentence-aligned, that means that source and target file contain one sentence per line which correspond to each other.

In [7]:
! head data/eng-epo/dev.eng

I made you something.
Twice two is four.
That hurts! Stop it!
It is too much for me. I need to slow down.
I never want to see him again.
At what hour was she born?
The traffic jam lasted one hour.
After dinner, we played cards till eleven.
I doubt if he is a lawyer.
Who's on duty today?


In [8]:
! head data/eng-epo/dev.epo

Mi faris ion por vi.
Du oble du faras kvar.
Tio suferigas min! Ĉesu!
Estas tro multe por mi. Mi devas malrapidiĝi.
Mi volas neniam plu vidi lin.
Je kioma horo ŝi naskiĝis?
La trafikmalfluo daŭris unu horon.
Post la vespermanĝo ni kartludis ĝis la dudek tria.
Mi dubas ĉu li estas advokato.
Kiu deĵoras hodiaŭ?


In [9]:
! wc -l data/eng-epo/*

    1000 data/eng-epo/dev.eng
    1000 data/eng-epo/dev.epo
  235834 data/eng-epo/dev.id
  235834 data/eng-epo/dev.src
  235834 data/eng-epo/dev.trg
    1000 data/eng-epo/test.eng
    1000 data/eng-epo/test.epo
   10000 data/eng-epo/test.id
   10000 data/eng-epo/test.src
   10000 data/eng-epo/test.trg
  402180 data/eng-epo/train.eng
  402180 data/eng-epo/train.epo
    1389 data/eng-epo/train.id.gz
 1547251 total


## Subword model training

We will use the `subword_nmt` library to split words into subwords (BPE) according to their frequency in the training corpus.

In [10]:
import os

In [11]:
src_lang = 'epo'
trg_lang = 'eng'
bpe_size = 4000
datadir = '/content/data/eng-epo/'
name = f'{src_lang}_{trg_lang}_bpe{bpe_size}'


train_src_file = os.path.join(datadir, f'train.{src_lang}')
train_trg_file = os.path.join(datadir, f'train.{trg_lang}')
train_joint_file = os.path.join(datadir, f'train.{src_lang}-{trg_lang}')
dev_src_file = os.path.join(datadir, f'dev.{src_lang}')
dev_trg_file = os.path.join(datadir, f'dev.{trg_lang}')
test_src_file = os.path.join(datadir, f'test.{src_lang}')
test_trg_file = os.path.join(datadir, f'test.{trg_lang}')
src_files = {'train': train_src_file, 'dev': dev_src_file, 'test': test_src_file}
trg_files = {'train': train_trg_file, 'dev': dev_trg_file, 'test': test_trg_file}


vocab_src_file = os.path.join(datadir, f'vocab.{bpe_size}.{src_lang}')
vocab_trg_file = os.path.join(datadir, f'vocab.{bpe_size}.{trg_lang}')
bpe_file = os.path.join(datadir, f'bpe.codes.{bpe_size}')

Train a BPE model with 4000 symbols for both languages jointly.

In [12]:
! cat $train_src_file $train_trg_file > $train_joint_file

! subword-nmt learn-bpe \
  --input $train_joint_file \
  -s $bpe_size \
  -o $bpe_file

100% 4000/4000 [00:24<00:00, 161.94it/s]


This file contains the merges of character sequences that subwords are made of.

In [13]:
! head $bpe_file

#version: 0.2
t h
a n
o n
e r
i n
e n
s t
l a</w>
o r


We apply the learned BPE merges to training, development and test data.


In [14]:
src_bpe_files = {}
trg_bpe_files = {}
for split in ['train', 'dev', 'test']:
  src_input_file = src_files[split]
  trg_input_file = trg_files[split]
  src_output_file = src_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  trg_output_file = trg_input_file.replace(split, f'{split}.{bpe_size}.bpe')
  src_bpe_files[split] = src_output_file
  trg_bpe_files[split] = trg_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $src_input_file > $src_output_file

  ! subword-nmt apply-bpe \
    -c $bpe_file \
    < $trg_input_file > $trg_output_file


The subword-split data contains `@@ ` to indicate where words were split into subwords.

In [15]:
! head data/eng-epo/dev.4000.bpe.eng

I made you some@@ thing.
T@@ w@@ ice two is f@@ our@@ .
That h@@ ur@@ t@@ s! Stop it@@ !
It is too much for me. I need to s@@ low dow@@ n.
I never want to see him again@@ .
A@@ t what h@@ our was she bor@@ n@@ ?
The traf@@ fi@@ c jam last@@ ed one h@@ our@@ .
Af@@ ter d@@ in@@ n@@ er, we played c@@ ards til@@ l el@@ ev@@ en.
I d@@ ou@@ b@@ t if he is a law@@ y@@ er.
Wh@@ o@@ 's on d@@ ut@@ y t@@ od@@ ay@@ ?


In [16]:
! head data/eng-epo/dev.4000.bpe.epo

Mi faris ion por vi.
D@@ u o@@ ble du faras kvar@@ .
Tio sufer@@ igas min@@ ! Ĉes@@ u!
Estas tro multe por mi. Mi devas mal@@ rapi@@ di@@ ĝ@@ i.
Mi volas neniam plu vidi lin.
J@@ e ki@@ om@@ a h@@ oro ŝi n@@ aski@@ ĝ@@ is?
La tra@@ fik@@ mal@@ fl@@ uo daŭ@@ ris unu hor@@ on.
Post la vesper@@ man@@ ĝo ni kart@@ lud@@ is ĝis la dudek tri@@ a.
Mi du@@ b@@ as ĉu li estas ad@@ vok@@ at@@ o.
Kiu de@@ ĵ@@ or@@ as hodi@@ aŭ@@ ?


## Prepare the vocabulary

From the pre-processed training data, we extract the final vocabulary for the translation model. It should contain all subwords needed for representing the source and target training data.

In [17]:
! wget https://raw.githubusercontent.com/joeynmt/joeynmt/1.5/scripts/build_vocab.py

--2022-09-04 07:57:40--  https://raw.githubusercontent.com/joeynmt/joeynmt/1.5/scripts/build_vocab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2034 (2.0K) [text/plain]
Saving to: ‘build_vocab.py’

build_vocab.py      100%[===================>]   1.99K  --.-KB/s    in 0s      

2022-09-04 07:57:40 (35.6 MB/s) - ‘build_vocab.py’ saved [2034/2034]



In [18]:
vocab_src_file = src_bpe_files['train']
vocab_trg_file = trg_bpe_files['train']
bpe_vocab_file = os.path.join(datadir, f'joint.{bpe_size}bpe.vocab')

! python build_vocab.py  \
  $vocab_src_file $vocab_trg_file \
  --output_path $bpe_vocab_file

# Model configuration

Joey NMT reads model and training hyperparameters from a configuration file. We're generating this now to configure paths in the appropriate places. 

The configuration below builds a small Transformer model with shared embeddings between source and target language on the base of the subword vocabularies created above.

In [19]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{datadir}/train.{bpe_size}.bpe"
    dev:   "{datadir}/dev.{bpe_size}.bpe"
    test:  "{datadir}/test.{bpe_size}.bpe"
    level: "bpe"
    lowercase: False                
    max_sent_length: 30             # Extend to longer sentences.
    src_vocab: "{vocab_src_file}"
    trg_vocab: "{vocab_trg_file}"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "intl"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # Alternative: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2000          # Set to at least once per epoch.
    logging_freq: 200
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True       # Requires joint vocabulary.
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # Increase to 512 for larger data.
        ff_size: 1024            # Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, source_language=src_lang, target_language=trg_lang,
           datadir=datadir, vocab_src_file=bpe_vocab_file, 
           vocab_trg_file=bpe_vocab_file, bpe_size=bpe_size)
with open("transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Training

This will take a while. The log reports the training process, look out for the prints of example translations and the BLEU evaluation scores to get an impression of the current quality. 

The log is also stored in the model directory within this runtime (inspect files in the menu on the left). There you can also find a summary report of all validations. We'll also use TensorBoard to visualize the training progress on the go. This requires enabling Cookies in the browser.

After 12h at the latest, Colab will disconnect, so to make sure you're progress is not lost, download the checkpoints from the model directory from time to time. You'll later be able to reload them if model hyperparameters match.

In [20]:
# Load the TensorBoard notebook extension. It will be empty at first.
%load_ext tensorboard

In [ ]:
%tensorboard --logdir models/epo_eng_bpe4000_transformer/tensorboard 

In [26]:
!python -m joeynmt train transformer_epo_eng_bpe4000.yaml

2022-09-04 08:04:38,110 - INFO - root - Hello! This is Joey-NMT (version 1.5).
2022-09-04 08:04:38,123 - INFO - joeynmt.data - Loading training data...
2022-09-04 08:04:43,183 - INFO - joeynmt.data - Building vocabulary...
2022-09-04 08:04:43,497 - INFO - joeynmt.data - Loading dev data...
2022-09-04 08:04:43,522 - INFO - joeynmt.data - Loading test data...
2022-09-04 08:04:44,235 - INFO - joeynmt.data - Data loaded.
2022-09-04 08:04:44,235 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 08:04:44,434 - INFO - joeynmt.model - Enc-dec model built.
2022-09-04 08:04:48,423 - INFO - joeynmt.training - Total params: 12223488
2022-09-04 08:04:52,040 - INFO - joeynmt.helpers -                           cfg.name : epo_eng_bpe4000_transformer
2022-09-04 08:04:52,040 - INFO - joeynmt.helpers -                       cfg.data.src : epo
2022-09-04 08:04:52,040 - INFO - joeynmt.helpers -                       cfg.data.trg : eng
2022-09-04 08:04:52,040 - INFO - joeynmt.helpers

## Continue training after interruption

To continue after an interruption, the configuration needs to be modified in 2 places: 
1. `load_model` to point to the checkpoint to load.
2. `model_dir` to create a new directory.


In [27]:
ckpt_number = 'latest' #2000
reload_config = config.replace(
    f'#load_model: "models/{name}_transformer/1.ckpt"', f'load_model: "models/{name}_transformer/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/{name}_transformer"', f'model_dir: "models/{name}_transformer_continued"')
with open("transformer_{name}_reload.yaml".format(name=name),'w') as f:
    f.write(reload_config)

Joey NMT then picks up training from there.

In [28]:
!python -m joeynmt train transformer_epo_eng_bpe4000_reload.yaml

2022-09-04 08:18:24,268 - INFO - root - Hello! This is Joey-NMT (version 1.5).
2022-09-04 08:18:24,304 - INFO - joeynmt.data - Loading training data...
2022-09-04 08:18:29,488 - INFO - joeynmt.data - Building vocabulary...
2022-09-04 08:18:29,811 - INFO - joeynmt.data - Loading dev data...
2022-09-04 08:18:29,835 - INFO - joeynmt.data - Loading test data...
2022-09-04 08:18:30,594 - INFO - joeynmt.data - Data loaded.
2022-09-04 08:18:30,594 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 08:18:30,804 - INFO - joeynmt.model - Enc-dec model built.
2022-09-04 08:18:32,550 - INFO - joeynmt.training - Total params: 12223488
2022-09-04 08:18:36,095 - INFO - joeynmt.training - Loading model from models/epo_eng_bpe4000_transformer/latest.ckpt
2022-09-04 08:18:36,327 - INFO - joeynmt.helpers -                           cfg.name : epo_eng_bpe4000_transformer
2022-09-04 08:18:36,328 - INFO - joeynmt.helpers -                       cfg.data.src : epo
2022-09-04 08:18:36,32

## Let's Translate!

The `test` mode can be used to translate (and evaluate on) the test set specified in the configuration. We usually do this only once after we've tuned hyperparameters on the dev set.

In [29]:
!python -m joeynmt test models/epo_eng_bpe4000_transformer_continued/config.yaml

2022-09-04 08:36:47,258 - INFO - root - Hello! This is Joey-NMT (version 1.5).
2022-09-04 08:36:47,259 - INFO - joeynmt.data - Building vocabulary...
2022-09-04 08:36:47,869 - INFO - joeynmt.data - Loading dev data...
2022-09-04 08:36:47,881 - INFO - joeynmt.data - Loading test data...
2022-09-04 08:36:47,890 - INFO - joeynmt.data - Data loaded.
2022-09-04 08:36:47,943 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2022-09-04 08:36:47,944 - INFO - joeynmt.prediction - Loading model from models/epo_eng_bpe4000_transformer_continued/10000.ckpt
2022-09-04 08:36:51,676 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 08:36:51,880 - INFO - joeynmt.model - Enc-dec model built.
2022-09-04 08:36:51,950 - INFO - joeynmt.prediction - Decoding on dev set (/content/data/eng-epo//dev.4000.bpe.eng)...
2022-09-04 08:37:05,451 - INFO - joeynmt.prediction -  dev bleu[intl]:  15.11 [Beam search decoding with beam size = 5 and alpha = {be

The `translate` mode is more interactive and takes prompts to translate interactively. Warning: it requires applying the same pre-processing steps to the new input as you've applied before model training (i.e. splitting into subwords).

In [30]:
from subword_nmt import apply_bpe

with open(bpe_file, "r") as merge_file:
  bpe = apply_bpe.BPE(codes=merge_file)

preprocess = lambda x: bpe.process_line(x.strip())

In [31]:
my_sentence = 'Esperanto, origine la Lingvo Internacia, estas la plej disvastiĝinta internacia planlingvo.'   # From https://eo.wikipedia.org/wiki/Esperanto

In [32]:
preprocess(my_sentence)

'E@@ sper@@ ant@@ o, or@@ ig@@ ine la L@@ ingv@@ o In@@ tern@@ aci@@ a, estas la plej dis@@ v@@ ast@@ iĝ@@ inta intern@@ ac@@ ia plan@@ lingv@@ o.'

Copy the above pre-processed sentence into the field when prompted below. Stop the cell to leave the interactive mode.

In [33]:
!python -m joeynmt translate models/epo_eng_bpe4000_transformer_continued/config.yaml

2022-09-04 08:37:42,784 - INFO - root - Hello! This is Joey-NMT (version 1.5).
2022-09-04 08:37:43,138 - INFO - joeynmt.prediction - Loading model from models/epo_eng_bpe4000_transformer_continued/10000.ckpt
2022-09-04 08:37:46,836 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 08:37:47,048 - INFO - joeynmt.model - Enc-dec model built.

Please enter a source sentence (pre-processed): 
E@@ sper@@ ant@@ o, or@@ ig@@ ine la L@@ ingv@@ o In@@ tern@@ aci@@ a, estas la plej dis@@ v@@ ast@@ iĝ@@ inta intern@@ ac@@ ia plan@@ lingv@@ o.
JoeyNMT: Hypotheses ranked by score
JoeyNMT #1: Esperantant, the Language Internal Internation is the most increated language.

Please enter a source sentence (pre-processed): 

Bye.
^C


You can also get the n-best hypotheses (up to the size of the beam, in our example 5), not only the highest scoring one. The better your model gets, the more interesting should the alternatives be.


In [34]:
!python -m joeynmt translate models/epo_eng_bpe4000_transformer_continued/config.yaml -n 3

2022-09-04 08:38:10,660 - INFO - root - Hello! This is Joey-NMT (version 1.5).
2022-09-04 08:38:11,018 - INFO - joeynmt.prediction - Loading model from models/epo_eng_bpe4000_transformer_continued/10000.ckpt
2022-09-04 08:38:14,730 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-04 08:38:14,936 - INFO - joeynmt.model - Enc-dec model built.

Please enter a source sentence (pre-processed): 
E@@ sper@@ ant@@ o, or@@ ig@@ ine la L@@ ingv@@ o In@@ tern@@ aci@@ a, estas la plej dis@@ v@@ ast@@ iĝ@@ inta intern@@ ac@@ ia plan@@ lingv@@ o.
JoeyNMT: Hypotheses ranked by score
JoeyNMT #1: Esperantant, the Language Internal Internation is the most increated language.
JoeyNMT #2: Esperantant, the Language Internal Internation is the most created language.
JoeyNMT #3: Esperantant, the Language Internal Internation is the most most increated language.

Please enter a source sentence (pre-processed): 

Bye.
^C
